In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
news_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/message_classification/news_cleaned.csv")
news_data

,text,finance
0,She left her husband. He killed their children...,0
1,Of course it has a song.,0
2,The actor and his longtime girlfriend Anna Ebe...,0
3,The actor gives Dems an ass-kicking for not fi...,0
4,"The ""Dietland"" actress said using the bags is ...",0
...,...,...
124984,NaN,0
124985,I often hear people describe cooking for one a...,0
124986,NaN,0
124987,NaN,0


In [4]:
news_data.isnull().sum()

text       19590
finance        0
dtype: int64

In [5]:
news_data.dropna(inplace=True)
news_data.reset_index(drop=True, inplace=True)

In [6]:
measurer = np.vectorize(len)
measurer(news_data['text'].astype(str)).max(axis=0)

1472

In [7]:
low = np.vectorize(str.lower)
news_data['text'] = low(news_data['text'].astype(str))

In [8]:
!pip3 install transformers 

     |████████████████████████████████| 2.5MB 8.3MB/s 
     |████████████████████████████████| 901kB 29.8MB/s 
     |████████████████████████████████| 3.3MB 54.1MB/s 


In [9]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", do_lower_case=True)

In [11]:
from sklearn.model_selection import train_test_split

news_data['text'] = news_data['text'].map(str)
X_train, X_test, y_train, y_test = train_test_split(news_data['text'], news_data['finance'], test_size=0.2)

In [12]:
X_train = X_train.tolist()
X_test = X_test.tolist()

In [13]:
max_length = 512

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

In [14]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, y_train.tolist())
valid_dataset = NewsGroupsDataset(valid_encodings, y_test.tolist())

In [15]:
torch.cuda.is_available()

True

In [16]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [18]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Colab Notebooks/data/message_classification/results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [19]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [20]:
trainer.train()

***** Running training *****
  Num examples = 84319
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 15810


Step,Training Loss,Validation Loss,Accuracy
200,0.199400,0.138833,0.969450
400,0.130300,0.090969,0.970683
600,0.118700,0.105793,0.969450


***** Running Evaluation *****
  Num examples = 21080
  Batch size = 20
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/data/message_classification/results/checkpoint-200
Configuration saved in /content/drive/MyDrive/Colab Notebooks/data/message_classification/results/checkpoint-200/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/data/message_classification/results/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21080
  Batch size = 20
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/data/message_classification/results/checkpoint-400
Configuration saved in /content/drive/MyDrive/Colab Notebooks/data/message_classification/results/checkpoint-400/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/data/message_classification/results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21080
  Batch size = 20
Saving model checkpoint to /content/dr

KeyboardInterrupt: ignored

Тут меня выкинул колаб